# Event linker

Where we link people and vehicles.

In [1]:
import pandas as pd
import numpy as np
import os
import json
import xml.etree.ElementTree as ET
import networkx as nx
import matplotlib.pyplot as plt


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
import network as net
n = net.network()
n.set_path('../../pop10k-eh4-qsim2-100it/output_network.xml.gz')
n.status()

n.load_nodes()
n.load_links()
n.join_network()
network = n.return_network()

#display(net.head())


AttributeError: module 'network' has no attribute 'network'

In [ ]:
display(network.tail())

,from,to,length,freespeed,capacity,permlanes,oneway,link,x_from,y_from,x_to,y_to,link_modes
72480,pt_U1998Z2P,pt_U1998Z2P,1.0,20.0,9999.0,1.0,1,pt_U1998Z2P,744447.8565812121,1043371.591993448,744447.8565812121,1043371.591993448,"artificial,stopFacilityLink,funicular"
72481,pt_U1998Z2P,pt_U1999Z2P,186.71831472037152,4.0,9999.0,1.0,1,pt_U1998Z2P_pt_U1999Z2P,744447.8565812121,1043371.591993448,744633.4534876784,1043392.0252350449,"artificial,funicular,rail"
72482,pt_U1999Z1P,pt_U1999Z1P,1.0,20.0,9999.0,1.0,1,pt_U1999Z1P,744642.1120766306,1043376.2352732904,744642.1120766306,1043376.2352732904,"artificial,stopFacilityLink,funicular"
72483,pt_U1999Z1P,pt_U1998Z1P,187.06857836204415,4.0,9999.0,1.0,1,pt_U1999Z1P_pt_U1998Z1P,744642.1120766306,1043376.2352732904,744455.0980943104,1043371.7160446909,"artificial,funicular,rail"
72484,pt_U1999Z2P,pt_U1999Z2P,1.0,20.0,9999.0,1.0,1,pt_U1999Z2P,744633.4534876784,1043392.0252350449,744633.4534876784,1043392.0252350449,"artificial,stopFacilityLink,funicular"


In [4]:
people = pd.read_json("./../output/events/agents.json")

people.sort_values("id", kind="stable", inplace=True)
people.set_index("id", inplace=True)
display(people.head(2))

,events
id,
1,"[{'event_id': 40544, 'time': 25879.0, 'type': ..."
2,"[{'event_id': 53594, 'time': 19854.0, 'type': ..."


In [ ]:
def load_vehicles(vehicle_types, path):
    vehicle_dict = {}
    for veh_type in vehicle_types:
        df = pd.DataFrame()
        df = pd.read_json(path+veh_type+".json")
        df.sort_values("id", kind="stable", inplace=True)
        df.set_index("id", inplace=True)
        #link coords
        vehicle_dict[str(veh_type)] = df.copy()
        print("Vehicle", veh_type, len(df.index.unique()))

    return vehicle_dict

In [ ]:
#transport = load_vehicles(["car","subway","bus","tram","ferry","rail","funicular"],"./output/events/")
transport = load_vehicles(["car","subway","bus","tram","funicular"],"./../output/events/")

## Linking people and vehicles
#link cars when entering
#link mhd when waiting

In [ ]:
def append_vehicles(df, person_id, vehicle_ids, verbal=False):
    events = pd.DataFrame()
    for v_id in vehicle_ids:
        if v_id is not None:
            veh_events = pd.DataFrame()
            if(verbal):
                print("Vehicle id:",v_id)

            if(str(v_id).isnumeric()):
                if(verbal):
                    print("\tVehicle type:", "car")
                car_row = transport['car'].loc[int(v_id)]
                car = pd.DataFrame.from_dict(car_row["events"])

                starts = df.iloc[np.where(df.vehicle_id == v_id)].loc[np.where(df.type == "PersonEntersVehicle")].time.to_list()
                ends = df.iloc[np.where(df.vehicle_id == v_id)].loc[np.where(df.type == "PersonLeavesVehicle")].time.to_list()

                car["vehicle_id"] = v_id
                for start,dest in zip(starts,ends):
                    if(verbal):
                        print(start, dest)
                        #display(car.iloc[np.where((car["time"] >= start) & (car["time"]<= dest))])
                    veh_events = veh_events.append(car.iloc[np.where((car["time"] >= start) & (car["time"]<= dest))])
            elif  v_id.split('_')[-1] in transport.keys():
                veh_type = v_id.split('_')[-1]
                if(verbal):
                    print("\tVehicle type:", veh_type)
                veh_row = transport[veh_type].loc[v_id]
                vehicle = pd.DataFrame.from_dict(veh_row["events"])
                vehicle["vehicle_id"] = v_id

                #tohle je spatne # tak uz dobry
                a = df.iloc[np.where(df.vehicle_id == v_id)] #pick start and end points of vehicle interaction(s)
                starts = a.iloc[np.where(a.type == "PersonEntersVehicle")].time.to_list()
                ends = a.iloc[np.where(a.type == "PersonLeavesVehicle")].time.to_list()
                for start,dest in zip(starts,ends):
                    if(verbal):
                        print(start, dest)
                        display(vehicle.iloc[np.where((vehicle["time"] >= start) & (vehicle["time"]<= dest))])
                    veh_events = veh_events.append(vehicle.iloc[np.where((vehicle["time"] >= start) & (vehicle["time"]<= dest))])
                
            #drop events
            if('type' in veh_events.columns):
                drop_idx = veh_events[
                            ((veh_events['type'] == "PersonEntersVehicle") & (veh_events['person_id'] != str(person_id))) | #
                            ((veh_events['type'] == "PersonLeavesVehicle") & (veh_events['person_id'] != str(person_id))) | #
                            #(veh_events['type'] == "departure") | 
                            #(veh_events['type'] == "arrival") | 
                            (veh_events['type'] == "vehicle leaves traffic") | 
                            (veh_events['type'] == "vehicle enters traffic") |
                            (veh_events['type'] == "left link")
                            ].index      
                veh_events.drop(drop_idx, inplace=True)
            #display(veh_events.type.unique())  
            #print("person_id",person_id, veh_events.person_id.unique())
            #display(veh_events)  
            events = events.append(veh_events)
    df = df.append(events, ignore_index=True)
    df = df.sort_values(["time"], kind="stable") #, "type"
    return df

In [ ]:
chunk_size = 500
chunk = pd.DataFrame()

last_id = 0
last_chunk = 0
chunk_no = last_chunk

for id, row in people.iterrows():
    df = pd.DataFrame.from_dict(row["events"])
    vehicle_ids = [ x for x in list(df.vehicle_id.unique())] #veh ids
    #print("Vehicle ids:", vehicle_ids)
    df = append_vehicles(df, id, vehicle_ids, verbal=False)
    m = df.reset_index()
    m.drop(columns=["index"], inplace=True)
    drop_idx = m[
        #(m['type'] == "departure") | 
        #(m['type'] == "arrival") | 
        (m['type'] == "vehicle leaves traffic") | 
        (m['type'] == "vehicle enters traffic") |
        (m['type'] == "left link")
        ].index
    m.drop(drop_idx, inplace=True)

    #merge with links
    ml = m.join(network.set_index("link"), on='link').fillna(value=np.nan)
    ml.drop(["from","to","length","event_id","permlanes",'capacity','link_modes'], axis=1, inplace=True)
    #ml["duration"] = ml.time.diff().replace(np.nan, 0)
    ml["person_id"] = id
    chunk = chunk.append(ml)
    if(id % chunk_size == 0):
        chunk.reset_index()
        chunk.to_csv("./../output/plans/"+str(chunk_no)+".csv")
        chunk_no +=1
        chunk = pd.DataFrame()
        #break
    vehicle_ids = []

### chunk operations

In [ ]:
#vehicles.head(5)

In [ ]:
car_ids = np.where(chunk['vehicle_id']==10.0)
vehicle = chunk.iloc[car_ids]
sharers = chunk.loc[chunk['person_id'].isin(vehicle.person_id.unique())]
sharers.actType.unique()
print("Car is used by: ",vehicle.person_id.unique())

In [ ]:
homes = sharers.iloc[np.where((sharers.actType == "home"))]
display(homes["coords_from"])

jobs = sharers.iloc[np.where((sharers.actType == "work"))]
display(jobs["coords_from"])